In [ ]:
import os

import pandas as pd 
import random 
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image 
import glob 


import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



from tqdm.notebook import tqdm
import csv

In [ ]:
def train_net(net, criterion, optimizer, train_dl, val_dl, N_EPOCHS):

    epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc = [], [], [], []

    for e in range(N_EPOCHS):

        ### TRAINING LOOP
        running_loss = 0
        running_accuracy = 0

        # Put the network in training mode
        net.train()

        for i, batch in enumerate(tqdm(train_dl)):

            # Get a batch from the dataloader
            x = batch[0]
            labels = batch[1]

            # Move the batch to GPU
            x = x.cuda()
            labels = labels.cuda()

            # Compute the network output
            y = net(x)

            # Compute the loss
            loss = criterion(y, labels)

            # Reset the gradients
            optimizer.zero_grad()

            # Compute the gradients
            loss.backward()

            # Apply one step of the descent algorithm to update the weights
            optimizer.step()

            # Compute some statistics
            with torch.no_grad(): # disables gradient calculations
                running_loss += loss.item()
                running_accuracy += (y.max(1)[1] == labels).sum().item()

        print("EPOCH:", e,"Training accuracy:", running_accuracy/float(len(cd_train_set)),
              "Training loss:", running_loss/float(len(cd_train_set)))

        epoch_loss.append(running_loss/len(cd_train_set))
        epoch_acc.append(running_accuracy/len(cd_train_set))

        ### VALIDATION LOOP
        # Put the network in validation mode
        net.eval()

        running_val_loss = 0
        running_val_accuracy = 0

        for i, batch in enumerate(tqdm(val_dl)):

            with torch.no_grad(): # disables gradient calculations
                # Get a batch from the dataloader
                x = batch[0]
                labels = batch[1]

                # Move the batch to GPU
                x = x.cuda()
                labels = labels.cuda()

                # Compute the network output
                y = net(x)

                # Compute the loss
                loss = criterion(y, labels)

                running_val_loss += loss.item()
                running_val_accuracy += (y.max(1)[1] == labels).sum().item()

        print("EPOCH:", e,"Validation accuracy:", running_val_accuracy/float(len(cd_val_set)),
              "Validation loss:", running_val_loss/float(len(cd_val_set)))

        epoch_val_loss.append(running_val_loss/len(cd_val_set))
        epoch_val_acc.append(running_val_accuracy/len(cd_val_set))
        
    return net

In [ ]:
pip install -U torchvision

In [ ]:
def efficientnet_b5():

    LEARNING_RATE = 0.0004
    
    net = torchvision.models.efficientnet_b5(pretrained=True)


    # Move model to the GPU
    net = net.cuda()

    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss().cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, criterion, optimizer

In [ ]:
net, criterion, optimizer =efficientnet_b5()

convert = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
ds = torchvision.datasets.ImageFolder("../input/polytech-nice-data-science-course-2021/polytech/train", transform = convert)

validation_split = 0.2
N_val_samples = round(validation_split * len(ds))

## Split into two Subset
cd_train_set, cd_val_set = torch.utils.data.random_split(ds, [len(ds) - N_val_samples, N_val_samples])
train_dl = torch.utils.data.DataLoader(cd_train_set, batch_size=32, shuffle=True, num_workers=2)
val_dl = torch.utils.data.DataLoader(cd_val_set, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
net_train = train_net(net, criterion, optimizer, train_dl, val_dl, N_EPOCHS=10)

In [ ]:
import glob
import os
from PIL import Image


classes=[]
image_name=[]
#classes=cd_train_set.classes

names = glob.glob("/kaggle/input/polytech-nice-data-science-course-2021/polytech/test/*.jpg")

N = len(names)

#test = names[1442]

#test[68:]

#basename = os.path.basename(test)
#file_name = os.path.splitext(basename)[0]

#file_name+".jpg"

for i in tqdm(range(N)):
    
    img = Image.open(names[i]).convert("RGB")
    y = nn.functional.softmax(net(convert(img).unsqueeze(0).cuda()), dim=1).max(1)[1].item()
    
    class_idx = ds.class_to_idx
    res = list(class_idx.keys())[list(class_idx.values()).index(y)]
    
    res = int(res)
    classes.append(res)

    basename = os.path.basename(names[i])
    file_name = os.path.splitext(basename)[0]
    image_name.append(file_name +".jpg")
    
  

output= pd.DataFrame(
    {'image_name': image_name,
     'class': classes,
    })  

In [ ]:
print(output)

In [ ]:
output.to_csv('submission_efficientnetb2.csv', index=False, sep=",")